## STOCK PREDICTION WEB APP

This project allows the prediction of the stock values of different companies through the Facebook Prophet tool, which allows an automated and more efficient prediction for the implementation of the models. To visualize the data, a web application built with the streamlit library was used, which allows the creation of apps in a fast and simple way.

# Required libraries

Install the following libraries:

In [1]:
%%capture
!pip install streamlit # Web app
!pip instlal prophet # Prediction
!pip install yfinance # Stock data collection
!pip install plotly # Data visualization
!pip install pyngrok # Web app host

Before implementing the model, a local host for the web application is created using the pyngrok library.

In [2]:
%%capture
!./ngrok authtokens 2DE2fhWhSFo9oYaxGJwIhSsyJyc_21DAbu3behebdMS4tysE5
from pyngrok import ngrok 
public_url = ngrok.connect(port='8501')

The implementation of the model is done in a single cell, this so that the host to the web application.

In [3]:
#Save the script in a Python file
%%writefile streamlit_app.py 

#Libraries 
import streamlit as st
import yfinance as yf
from datetime import date
from prophet import Prophet
from prophet.plot import plot_plotly
from plotly import graph_objs as go

#Prediction dates
START = '2015-01-01'
TODAY = date.today().strftime('%Y-%m-%d')

#Web app title
st.title('Stock Prediction App')

#Selected stocks to predict
stocks = ('GOOG', 'MSFT', 'GME') #Add or change to any you want
selected_stock = st.selectbox('Select dataset for prediction', stocks)

#Years to predict
n_year = st.slider('Years of prediction:', 1, 4)
period = n_year * 365 

#Loading data 

def load_data(ticker):
    data = yf.download(ticker, START, TODAY)
    data.reset_index(inplace = True)
    return data

data = load_data(selected_stock)

#Displaying row data

st.subheader('Raw data')
st.write(data.tail())

def plot_raw_data():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = data['Date'], y = data['Open'], name = 'stock_open'))
    fig.add_trace(go.Scatter(x = data['Date'], y = data['Close'], name = 'stock_close'))
    fig.layout.update(title_text = 'Time Series Data', xaxis_rangeslider_visible = True)
    st.plotly_chart(fig)

plot_raw_data()

#Forecasting

df_train = data[['Close','Date']]
df_train = df_train.rename(columns = {'Date': 'ds', 'Close': 'y'})

#Model implementation
m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods = period)
forecast = m.predict(future)

#Display of results
st.subheader('Forecast data')
st.write(forecast.tail())

fig1 = plot_plotly(m, forecast)
st.plotly_chart(fig1)

Writing streamlit_app.py


Once the model has been completed, proceed to upload the application to the host. Select the url that says: "your url is: "

In [4]:
!streamlit run /content/streamlit_app.py & npx localtunnel --port 8501    

2022-08-17 05:34:26.861 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.85.180.154:8501

npx: installed 22 in 3.793s
your url is: https://fruity-terms-stop-34-85-180-154.loca.lt
[*********************100%***********************]  1 of 1 completed
2022-08-17 05:34:44.376 Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
2022-08-17 05:34:44.411 input tempfile: /tmp/tmpsaigokm_/ud5e7q4w.json
2022-08-17 05:34:44.443 input tempfile: /tmp/tmpsaigokm_/t_yfe1k4.json
2022-08-17 05:34:44.444 idx 0
2022-08-17 05:34:44.444 running CmdStan, num_threads: None
2022-08-17 05:34:44.445 CmdStan args: ['/usr/local/lib/python3.7/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=58956', 'data', 'file=/tmp/tmpsaigokm_/ud5e7q4w.json', 'init=/tmp/tmpsaigokm_/t_yfe1k4.json', 'output', 'file=/tmp/tmpdo6zelm_/prophet_model-2022081